In [1]:
import numpy as np
import os
os.environ['NUMBA_THREADING_LAYER'] = 'omp' # my TBB version complains
os.environ['NUMBA_NUM_THREADS'] = '3'
os.environ['NUMBA_ENABLE_AVX'] = '1'
os.environ['NUMBA_CPU_NAME'] = 'generic'
os.environ['NUMBA_CPU_FEATURES'] = '+sse,+sse2,+avx,+avx2,+avx512f,+avx512dq'
#os.environ['NUMBA_CPU_FEATURES'] = '+sse,+sse2,+avx,+avx2'
import sys
sys.path.insert(0,'/Users/ban115/bolton/craco-python/src/')


import numba
from numba import njit, prange
from pylab import *
%matplotlib notebook

%load_ext autoreload
%autoreload 2


In [2]:
nant = 30
nbl = nant*(nant+1)//2
nt = 32
nchan = 24
nbeam = 36

vin = np.arange(nbl*nt*nchan*2, dtype=np.int16).reshape((nchan, nbl, nt, 2))
vavg = np.zeros((nchan, nbl, 2), dtype=np.float32)
ics = np.zeros((nt, nchan), dtype=np.float32)
cas = np.zeros((nt, nchan), dtype=np.float32)

vin.size

714240

In [3]:
# For type hint info see https://numba.pydata.org/numba-doc/0.12.2/tutorial_types.html
@njit(fastmath=True, parallel=True, locals={'vabs':numba.float32})
def make_averages(nant, vin, vavg, ics, cas):
    nchan,nbl, nt, _ = vin.shape
    
    for ic in range(nchan):
        ibl = 0
        for ia1 in range(nant):
            for ia2 in range(ia1, nant):
                #print(ia1, ia2, ibl, nbl)
                #assert ibl < nbl#, f'Invalid ibl {ia1} {ia2} {ibl} {nbl}'
                
                for it in range(nt):
                    v = vin[ic, ibl, it, :]
                    #v = np.array([0,1])
                    vabs = v[0]*v[0] + v[1]*v[1]
                    if ia1 == ia2:
                        ics[it, ic] += vabs
                    else:
                        cas[it, ic] += vabs

                    #print('out', ibl, ic, it)
                    vavg[ic, ibl, 0] += v[0]
                    vavg[ic, ibl, 1] += v[1]
                    
                    #print('done', ibl, ic, it)


                ibl += 1

    ics //= nant
    cas //= nbl
    vavg //= nt
            


In [4]:
for i in range(1):
    make_averages(nant, vin, vavg, ics, cas)

/data/seren-01/fast/ban115/build/venv3.6/lib/python3.6/site-packages/numba/core/typed_passes.py:327: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-3-34d199a97e29>", line 3:
@njit(fastmath=True, parallel=True, locals={'vabs':numba.float32})
def make_averages(nant, vin, vavg, ics, cas):
^

  state.func_ir.loc))


In [5]:
make_averages.parallel_diagnostics(level=4)


 
 Parallel Accelerator Optimizing:  Function make_averages, <ipython-
input-3-34d199a97e29> (2)  
No source available
--------------------------------- Fusing loops ---------------------------------
Attempting fusion of parallel loops (combines loops with similar properties)...
----------------------------- Before Optimisation ------------------------------
--------------------------------------------------------------------------------
------------------------------ After Optimisation ------------------------------
Parallel structure is already optimal.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 
---------------------------Loop invariant code motion---------------------------
Allocation hoisting:
No allocation hoisting found

Instruction hoisting:
No instruction hoisting found
--------------------------------------------------------------------------------


In [6]:
%timeit -n1 make_averages(nant, vin, vavg, ics, cas)

2.19 ms ± 74.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
vin.shape

(24, 465, 32, 2)

In [8]:
make_averages.inspect_types()

make_averages (int64, array(int16, 4d, C), array(float32, 3d, C), array(float32, 2d, C), array(float32, 2d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-3-34d199a97e29>
# --- LINE 2 --- 

@njit(fastmath=True, parallel=True, locals={'vabs':numba.float32})

# --- LINE 3 --- 

def make_averages(nant, vin, vavg, ics, cas):

    # --- LINE 4 --- 
    # label 0
    #   nant = arg(0, name=nant)  :: int64
    #   vin = arg(1, name=vin)  :: array(int16, 4d, C)
    #   vavg = arg(2, name=vavg)  :: array(float32, 3d, C)
    #   ics = arg(3, name=ics)  :: array(float32, 2d, C)
    #   cas = arg(4, name=cas)  :: array(float32, 2d, C)
    #   $0.2 = getattr(value=vin, attr=shape)  :: UniTuple(int64 x 4)
    #   $0.7 = exhaust_iter(value=$0.2, count=4)  :: UniTuple(int64 x 4)
    #   del $0.2
    #   $nchan.28 = static_getitem(value=$0.7, index=0, index_var=None, fn=<built-in function getitem>)  :: int64
    #   $nbl.29 = static_getitem(v

In [9]:
from craco.cardcapfile import  get_single_packet_dtype, NCHAN,get_indexes
%aimport craco.card_averager
from craco.card_averager import *
import craco.card_averager as avg

In [10]:
nfpga = 6
nbeam = 36
nc_per_fpga = 4
nt = 16
npkt = nbeam*nc_per_fpga
pktshape = (npkt, nt)
polsum = True
debughdr = True
dtype = get_single_packet_dtype(nbl, debughdr, polsum)
din_list = [np.zeros(pktshape, dtype=dtype) for i in range(nfpga)]
packets = [pkt for pkt in din_list]
packet_list = List()
[packet_list.append(self.dummy_packet if pkt is None else pkt) for pkt in packets]
_,_,auto_idxs,cross_idxs = get_indexes(nant)
din = np.array(din_list)
tscrunch = 4
data = din['data']
print(din.shape)
print(data.shape)


(6, 144, 16)
(6, 144, 16, 2, 465, 1, 2)


In [11]:
din['data'].shape

(6, 144, 16, 2, 465, 1, 2)

In [12]:
np.array?

In [13]:
%timeit np.array(din_list)

12.5 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit np.array(packets)

The slowest run took 6.70 times longer than the fastest. This could mean that an intermediate result is being cached.
24.8 ms ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit np.array(packet_list)

13.5 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
dout = avg.average0(din)

In [20]:
print(avg.average0(din).shape)
%timeit avg.average0(din)

(144, 16, 2, 465, 2)
39.9 ms ± 720 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
print(avg.average1(din).shape)
%timeit avg.average1(din)

(144, 465, 2)
13.6 ms ± 703 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit dout.astype(np.int16)

1.97 ms ± 31.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
(nfpga, npkt, nt1, nt2, nbl, npol, _) = data.shape
# This reshape takes 19 milliseconds!
%timeit dreshape =  din['data'].reshape(nfpga, npkt, 1, nt1*nt2, nbl, npol, 2)


18.2 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
print(avg.average2(din, tscrunch=32).shape)

%timeit avg.average2(din, tscrunch=4)

(144, 1, 465, 2)
33.1 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit average3(din, tscrunch=2)

34.3 ms ± 778 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit average4(din, tscrunch=2)

32.2 ms ± 905 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
data = din['data']
(nfpga, npkt, nt1, nt2, nbl, _, _) = data.shape
dshape = (npkt, nt1*nt2 // tscrunch, nbl, 2)
dout = np.zeros(dshape, dtype=np.float32)
%timeit avg.average4(din, tscrunch=tscrunch, dout=dout)

31.4 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit avg.average4(din, tscrunch=tscrunch, dout=None)

32.4 ms ± 979 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
import llvmlite.binding as llvm
llvm.set_option('', '--debug-only=loop-vectorize')

In [30]:
data.dtype

dtype('int16')

In [31]:
dout.dtype

dtype('float32')

In [32]:
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [33]:
avg.average5(data, tscrunch=tscrunch, dout=dout)
%timeit avg.average5(data, tscrunch=tscrunch, dout=dout)

81.5 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%autoreload average6

average6(din['data'], tscrunch=tscrunch, dout=dout)
%timeit average6(din['data'], tscrunch=tscrunch, dout=dout)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(int16, 7d, A), Literal[str](data))
 
There are 22 candidate implementations:
  - Of which 20 did not match due to:
  Overload of function 'getitem': File: <numerous>: Line N/A.
    With argument(s): '(array(int16, 7d, A), unicode_type)':
   No match.
  - Of which 1 did not match due to:
  Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 162.
    With argument(s): '(array(int16, 7d, A), unicode_type)':
   Rejected as the implementation raised a specific error:
     TypeError: unsupported array index type unicode_type in [unicode_type]
  raised from /data/seren-01/fast/ban115/build/venv3.6/lib/python3.6/site-packages/numba/core/typing/arraydecl.py:69
  - Of which 1 did not match due to:
  Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py

In [ ]:
dout.shape

In [ ]:
average6.inspect_types()

In [ ]:
intermediate_dout = np.zeros(din['data'].shape, dtype=np.float32)
%timeit avg.average9(din, tscrunch=tscrunch, dout=intermediate_dout)

In [ ]:
dout.shape

In [ ]:
np.array(packets).shape

In [ ]:
packets[0].dtype

In [ ]:
packets[0].shape

In [ ]:
packets[0]['data'].shape

In [ ]:
(nibc, nt1, nt2, nbl, npol, _) = packets[0]['data'].shape
ntout = nt1 * nt2 // tscrunch
dout = np.zeros(packets[0]['data'].shape, dtype=np.float32)
dout1 = np.zeros((NBEAM*NCHAN,nt1*nt2//tscrunch,nbl,2), dtype=np.float32)

print(avg.average10(packets, dout=dout).shape)
%timeit avg.average10(packets, dout=dout)

In [ ]:
%timeit dout.reshape(144,32//tscrunch,tscrunch,465,2).mean(axis=2)

In [ ]:
%timeit dout.reshape(144,32//tscrunch,tscrunch,465,1,2).mean(axis=(2,4))

In [ ]:
%timeit dout1[:] = dout.reshape(144,32//tscrunch,tscrunch,465,1,2).mean(axis=(2,4))

In [ ]:
dout = np.zeros(packets[0]['data'].shape, dtype=np.float32)
print(avg.average11(packet_list,  dout=dout).shape)
%timeit avg.average11(packet_list,  dout=dout)

In [ ]:
ntout

In [ ]:
dout2 = avg.average10(packets, None)
print(avg.average13(packets, tscrunch, dout1, dout2).shape)
%timeit avg.average13(packets, tscrunch, dout1, dout2).shape

In [ ]:
vis_dout = np.zeros((NBEAM,len(cross_idxs),NCHAN,ntout,2), dtype=np.float32)
avg.vis_reshape(dout1, cross_idxs, vis_dout).shape
%timeit avg.vis_reshape(dout1, cross_idxs, vis_dout)

In [ ]:
avg.average_pkts_and_reshape(packets, tscrunch, vis_dout, cross_idxs, dout1, dout2).shape
%timeit avg.average_pkts_and_reshape(packets, tscrunch, vis_dout, cross_idxs, d1=None, d2=None).shape

# number is super slow even with normal numpy arrays

In [ ]:
%timeit dout[:] = 0

In [ ]:
%timeit dout * 3

In [ ]:
dout = np.zeros((144, 16, 2, 465, 1, 2), dtype=np.float32)

In [ ]:
print(numba.__version__)

In [ ]:
@njit
def numba_clear(d):
    d[:] = 0
    

In [ ]:
%timeit dout[:]  = 0

In [ ]:
numba_clear(dout)
%timeit numba_clear(dout)

In [ ]:
dout_flat = dout.flatten()
numba_clear(dout_flat)
%timeit numba_clear(dout_flat)

In [ ]:
packets[0]['data'].reshape(144,32,465,2).

# How about the card averager

In [ ]:
from craco import card_averager

In [ ]:
from numba.typed import List
data = List()
%timeit [data.append(self.dummy_packet if pkt is None else pkt) for pkt in packets]


In [45]:
d = packets[0]
d['data'].shape

(144, 16, 2, 465, 1, 2)

In [46]:
d['data'][:,:,:,auto_idxs,:,0].shape

(30, 144, 16, 2, 1)

In [48]:
np.array(packets)['data'][:,:,:,:,auto_idxs,0,0].shape

(6, 144, 16, 2, 30)

In [49]:
avg.calc_ics(packets, auto_idxs).shape

(6, 144, 16, 2)

In [50]:
%timeit avg.calc_ics(packets, auto_idxs).shape

4.71 ms ± 325 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
from craco import card_averager

In [53]:

dummy_packet = packets[0].copy()
print(type(dummy_packet))

avger = card_averager.Averager(nbeam,nant,nc=24,nt=32,npol=1,vis_fscrunch=6, vis_tscrunch=4, dummy_packet=dummy_packet, cdtype=np.float32)


<class 'numpy.ndarray'>


/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:685: RuntimeWarning: invalid value encountered in true_divide
  variance = m2 / count #ill produce NAN where count=0
/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:694: RuntimeWarning: divide by zero encountered in true_divide
  scale = 1/stdev


In [55]:
valid = np.ones(6, dtype=bool)

In [58]:
%timeit avg.calc_and_reshape_ics(packets, auto_idxs, valid, avger.output['ics'])

4.96 ms ± 59.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
# current averager takes 270 milliseconds - which is just a bloody outrage - I mean seriously
%timeit avger.accumulate_packets(packets)

/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:690: RuntimeWarning: invalid value encountered in true_divide
  # not sure if I should use variance, or sample variance, let's use variance
/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:699: RuntimeWarning: divide by zero encountered in true_divide
  np.savez(fout, mean=mean,count=count,scount=self.count,variance=variance,stdev=stdev,offset=offset,scale=scale)


33.2 ms ± 794 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
avger = card_averager.Averager(nbeam,nant,24,32,1,6,4, dummy_packet=dummy_packet, cdtype=np.float32)

/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:690: RuntimeWarning: invalid value encountered in true_divide
  variance = m2 / count #ill produce NAN where count=0
/data/seren-01/fast/ban115/build/craco-python/src/craco/card_averager.py:699: RuntimeWarning: divide by zero encountered in true_divide
  scale = 1/stdev


In [ ]:
avger.output.dtype

In [ ]:
avger.output.shape

In [ ]:
avger.output['vis'].shape

In [ ]:
from craco.card_averager import ibc2beamchan


_= average7(din, tscrunch=8, dout=avger.output['vis'])
%timeit average7(din, tscrunch=8, dout=avger.output['vis'])

In [ ]:


_= average8(din, tscrunch=8, dout=avger.output['vis'],nant=nant)
%timeit average8(din, tscrunch=8, dout=avger.output['vis'],nant=nant)

In [ ]:
avger.output['vis'].shape

In [ ]:
average2(din,tscrunch=8).shape

In [ ]:
def average_vis_and_reshape2(din, tscrunch, dout, auto_idxs, cross_idxs):
    '''
    Writes (beam,chan) order and removes autocorrelations
    Fxed fscrunch at 6
    This is horriffically slow in NUMPY - really, really bad
    '''
    data = din['data']
    (nfpga, npkt, nt1, nt2, nblall, _, _) = data.shape
    dout[:] = 0
    # HACK - set nbl to output NBL, which doesn't include autos, for now
    #nbl = dout.shape[1]
    ntout = nt1*nt2//tscrunch
    
    nbl = len(cross_idxs)
    nant = len(auto_idxs)
    dcross = data[:,:,:,:,cross_idxs,:,:]
    d = dcross.reshape(nfpga, npkt, ntout, tscrunch, nbl, 2).mean(axis=(0,3), dtype=np.float32)
    dcopy1 = d[:32*4,...].reshape(32,4,ntout,nbl,2).transpose(0,3,1,2,4)
    dcopy2 = d[32*4:,...].reshape(4,4,ntout,nbl,2).transpose(0,3,1,2,4)


    dout[0:32, :,:,:,:] = dcopy1
    dout[32:, :,:,:,:] = dcopy2

            
    return dout

print(average_vis_and_reshape2(din, tscrunch=4, dout=avger.output['vis'],auto_idxs=auto_idxs,cross_idxs=cross_idxs).shape)
%timeit average_vis_and_reshape2(din, tscrunch=4, dout=avger.output['vis'],auto_idxs=auto_idxs,cross_idxs=cross_idxs)

In [ ]:
avger.output['vis'].shape

In [ ]:
avger.output['cas'].shape

In [ ]:
din['data'].shape

In [ ]:
data = din['data'].astype(np.float32)
data.shape

In [ ]:
def amp1(data):
    return np.sqrt(data[...,0]**2 + data[...,1]**2)

%timeit amp1(data)

In [ ]:
from math import sqrt

@njit
def amp2(data, dout):
    (nfpga, npkt, nt1, nt2, nbl, _, _) = data.shape
    dout[:] = 0
    # HACK - set nbl to output NBL, which doesn't include autos, for now
    nbl = dout.shape[1]
    
    # dout shape
    # avger.output['vis'].shape
    # (36, 435, 4, 8, 2)
    
    for ifpga in range(nfpga):
        for ipkt in range(npkt):
            beam,chan = ibc2beamchan(ipkt)
            #print(beam,chan)
            for t1 in range(nt1):
                for t2 in range(nt2):
                    ttotal = t2 + t1*nt2
                    tout = ttotal // tscrunch
                    for ibl in range(nbl):
                        d0 = data[ifpga,ipkt,t1,t2,ibl,0,0]
                        d1 = data[ifpga,ipkt,t1,t2,ibl,0,1]

                        da = sqrt(d0**2 + d1**2)    
                        dout[ifpga,ipkt,t1,t2,ibl,0] = da
                        
    return dout
                            
    

In [ ]:
dout = np.zeros(data.shape[:-1], dtype=np.float32)
amp2(data, dout).shape
%timeit amp2(data, dout)

In [ ]:
data.shape

In [ ]:
avger.output['ics'].shape

In [ ]:
%timeit calc_ics(data, auto_idxs)

In [ ]:
dmean = calc_ics(data, auto_idxs)
dmean.shape

In [ ]:
icsout = avger.output['ics']

In [ ]:
icsout.shape

In [ ]:
(nfpga, npkt, nt1, nt2, nbl, npol, _) = data.shape
data.shape

In [ ]:
dmean[:,:32*4,:,:].reshape(nfpga,4,32,32).transpose(2,3,1,0).reshape(32,32,24).shape

In [ ]:
dmean[:,32*4:,:,:].reshape(nfpga,4,4,32).transpose(2,3,1,0).reshape(4,32,24).shape

In [ ]:
valid = np.zeros(6, dtype=bool)
%timeit calc_and_reshape_ics(data, auto_idxs, valid, avger.output['ics'])

In [ ]:
average